In [6]:
import os
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from openai import AzureOpenAI

#1. 환경 변수 로드(.env 파일 불러오기)
load_dotenv()

#2. api 키 불러오기
aoai_key = os.getenv("AZURE_OPENAI_API_KEY")
speech_key = os.getenv("SPEECH_KEY")

print(f"▶ OpenAI 키 확인: {aoai_key[:3]}***")
print(f"▶ Speech 키 확인: {speech_key[:3]}***")
print("✅ 준비 완료! 키가 안 보이면 .env 파일 위치나 오타를 확인해.")

▶ OpenAI 키 확인: 8K0***
▶ Speech 키 확인: 6Yb***
✅ 준비 완료! 키가 안 보이면 .env 파일 위치나 오타를 확인해.


In [12]:
# AZURE OPENAI 클라이언트 생성성
client = AzureOpenAI(
    api_key= os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
print(f"AI 모델{deployment_name}에게 질문하는 중...")

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "너는 친절한 영어 회화 튜터야."},
        {"role": "user", "content": "안녕하세요 오늘의 영어공부 시작합시다."}
    ]
)

#답변 받아오기
answer = response.choices[0].message.content
print(f"AI 모델의 답변: {answer}")

AI 모델gpt-4o-mini에게 질문하는 중...
AI 모델의 답변: 안녕하세요! 영어 공부를 시작해 봅시다. 어떤 주제에 대해 이야기하고 싶으신가요? 아니면 특정한 문법이나 어휘를 배우고 싶으신가요?


In [ ]:
#TTS(ai의 입) 설정 완료

speech_key = os.getenv("SPEECH_KEY")
speech_region = os.getenv("SPEECH_REGION")

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
speech_config.speech_synthesis_voice_name = "en-US-jennyNeural"

# 오디오 출력 설정
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
#합성기(Synthesizer) 생성
synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

print("말하기 테스트중..(스피커를 켜두세요.)")
text_to_speak = "Hello, I am ready to start your English study."

#말해!
result = synthesizer.speak_text_async(text_to_speak).get()

if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("말하기 성공!")
else:
    print(f"말하기 실패, 이유:{result.cancellation_details.reason_details}")

말하기 테스트중..(스피커를 켜두세요.)
말하기 성공!


In [18]:
#STT 설정 (ai의 귀)

#영어만 듣겠다고 설정.
speech_config.speech_recognition_language = "en-US"

#오디오 입력 설정(내 마이크)
audio_input = speechsdk.audio.AudioConfig(use_default_microphone=True)
recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

print(f"마이크 테스트! 3초안에 영어로 한마디 해보세요 (예: Hello).")

result = recognizer.recognize_once_async().get()

if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print(f"인식된 텍스트: {result.text}")
elif result.reason == speechsdk.ResultReason.NoMatch:
    print(f"인식 실패, 아무말도 못들었어.")
else:
    print(f"에러 발생: {result.cancellation_details.reason_details}")

마이크 테스트! 3초안에 영어로 한마디 해보세요 (예: Hello).
인식된 텍스트: Hello, nice to meet you. My name is Chang Song and I'm happy to know that.
